In [ ]:
from keras.callbacks import ModelCheckpoint , EarlyStopping ,TensorBoard
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import scipy.io
import keras
from keras_vggface.vggface import VGGFace
from keras.applications import VGG16
from keras import layers
from keras.models import Model
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

from keras.layers import Dense, Dropout, GlobalAveragePooling2D ,Flatten
import cv2
import numpy as np
from PIL import Image
import dlib
import random
np.random.seed(42)
from keras_vggface import utils

In [ ]:
pathTrain= "C:\\Users\\Shehab\\Desktop\\Finish\\Ethinicity\\IDG New"


datagenTrain = ImageDataGenerator(rescale=1./255,
                             validation_split=0.3 ,
                            rotation_range=20,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest',
                            preprocessing_function=utils.preprocess_input
                            )

train_generator = datagenTrain.flow_from_directory(
        pathTrain,
        target_size=(224, 224),
        batch_size=128,
        class_mode='categorical',
         shuffle=True,
         subset="training")

validation_generator = datagenTrain.flow_from_directory(
        pathTrain,
        target_size=(224, 224),
        batch_size=128,
        class_mode='categorical',
        subset="validation" ,
        shuffle=False)




In [ ]:
class_labels = list(train_generator.class_indices.keys())
class_counts = train_generator.classes

fig, ax = plt.subplots()
ax.hist(class_counts, bins=len(class_labels))
ax.set_xticks(range(len(class_labels)))
ax.set_xticklabels(class_labels, rotation=45, ha='right')
ax.set_xlabel('Class ')
ax.set_ylabel('Number of images')
plt.show()

In [ ]:
class_labels = list(validation_generator.class_indices.keys())
class_counts = validation_generator.classes

fig, ax = plt.subplots()
ax.hist(class_counts, bins=len(class_labels))
ax.set_xticks(range(len(class_labels)))
ax.set_xticklabels(class_labels, rotation=45, ha='right')
ax.set_xlabel('Class - Test Dataset')
ax.set_ylabel('Number of images')
plt.show()

In [ ]:
num_images = [299374, 310804, 269383, 316042 ,17878]
total_images = sum(num_images)
class_weights = {i: total_images/(len(num_images)*num_images[i]) for i in range(len(num_images))}

In [ ]:
vggface = VGGFace(model='vgg16' , include_top=False ,input_shape=(224, 224, 3))

for layer in vggface.layers:
    layer.trainable = False
    
x = vggface.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(5, activation='softmax')(x)

model = Model(inputs=vggface.input, outputs=output_layer)
model.summary()



In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpoint = ModelCheckpoint(filepath="CheckPoint.h5",
                             monitor='val_accuracy',
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1
                            )
early_stopping = EarlyStopping(monitor='val_loss', patience=5,verbose=1)
tensorboard = TensorBoard(log_dir="C:\\Users\\Shehab\\Desktop\\Ethnicity")


In [ ]:


config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 6} ) 
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config) 
keras.backend.set_session(sess)

with tf.device('/GPU:0'):
    history = model.fit(train_generator,
                              batch_size=128,
                              validation_data=validation_generator,
                              epochs=10,
                              callbacks=[early_stopping, checkpoint],
                              shuffle=False,
                              verbose=1,
                              validation_steps=validation_generator.samples // validation_generator.batch_size,
                              steps_per_epoch=train_generator.samples // train_generator.batch_size,
                                class_weight=class_weights
                             )

In [ ]:
train_loss = history.history['loss']
test_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
test_accuracy = history.history['val_accuracy']

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15,7))

ax = ax.ravel()

ax[0].plot(train_loss, label='Train Loss', color='royalblue', marker='o', markersize=5)
ax[0].plot(test_loss, label='Test Loss', color = 'orangered', marker='o', markersize=5)

ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)

ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)

ax[1].plot(train_accuracy, label='Train Accuracy', color='royalblue', marker='o', markersize=5)
ax[1].plot(test_accuracy, label='Test Accuracy', color='orangered', marker='o', markersize=5)

ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)

ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)

fig.suptitle(x=0.5, y=0.92, t="Lineplots showing loss and accuracy of CNN model by epochs", fontsize=16)

plt.savefig('final_cnn_loss_accuracy.png', bbox_inches='tight');

In [ ]:
model.evaluate(validation_generator)

In [ ]:
predictions = model.predict(validation_generator, steps=validation_generator.samples // 128 + 1)


In [ ]:
predicted_classes =[]
for pred in predictions:
    predicted_classes.append(np.argmax(pred))

    

In [ ]:
true_classes = validation_generator.classes

In [ ]:
cm = confusion_matrix(true_classes, predicted_classes)


In [ ]:

cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
target_names = ['caucassian', 'African', 'Indian', 'Asian' , 'middle east']
fig, ax = plt.subplots(figsize=(8, 8))
ax.matshow(cm_norm, cmap=plt.cm.Blues, alpha=0.3)
for i in range(cm_norm.shape[0]):
    for j in range(cm_norm.shape[1]):
        ax.text(x=j, y=i, s=format(cm_norm[i, j], '.2f'),
                ha='center', va='center', color='black', fontsize=14)
plt.xticks(np.arange(len(target_names)), target_names, fontsize=14)
plt.yticks(np.arange(len(target_names)), target_names, fontsize=14)
plt.xlabel('Predicted label', fontsize=14)
plt.ylabel('True label', fontsize=14)
plt.title('Confusion matrix', fontsize=16)
plt.show()